You are tasked with developing a movie recommendation system. You are given a list of movies (their names) and a list of similarities between movies (pairs of movies that are similar). You are also given a list of user's friends and for each friend a list of movies that he has already seen.

Your system should recommend one movie with the highest discussability and uniqueness. Discussability is the number of friends of user, who have already seen that movie. Uniqueness is 1 divided by the mean number of similar movies that the user's friends have already seen. So you should return the film with the highest number: F / S, where F = number of friends who have seen this movie, and S = mean of the number of similar movies seen for each friend.

In [8]:
### Test №1
### https://www.coursera.org/learn/algdata2/discussions/weeks/6/threads/LODjGJJAEeuw1Q42CpJ9yQ

movies = ["Parasite","1917","Ford v Ferrari","Jojo Rabbit","Joker"]

similarities = [["Parasite", "1917"],
                ["Parasite", "Jojo Rabbit"],
                ["Joker", "Ford v Ferrari"]]
### 1917 <-> Parasite <-> Jojo Rabbit

friends = [["Joker"],
            ["Joker","1917"],
            ["Joker"],
            ["Parasite"],
            ["1917"],
            ["Jojo Rabbit", "Joker"]]

### Answer is "1917"

In [2]:
### Test №2
movies = {'Group0_1', 'Group0_2', 'Group0_3', 'Group2_2', 'Group1_1',
          'Group2_1','Group0_4', 'Group1_3', 'Group0_5', 'Group1_2'}

similarities = [["Group0_1", "Group0_2"],
                ["Group0_2", "Group0_3"],
                ["Group1_1", "Group1_2"],
                ["Group0_3", "Group0_4"],
               ["Group0_5", "Group0_4"],
               ["Group1_2", "Group1_3"],
               ["Group2_1", "Group2_2"]]

friends = [['Group0_2', 'Group0_3'],
            ['Group0_4', 'Group1_3', 'Group0_5', 'Group1_2'],
            ['Group0_2', 'Group1_2'],
            ['Group0_3', 'Group2_2', 'Group1_1', 'Group1_3'],
            ['Group0_1', 'Group0_3', 'Group1_1', 'Group1_2', 'Group0_4'],
            ['Group2_1','Group0_4']]

### Answer is "Group0_3" or "Group0_4"

In [6]:
### Test №3
movies = ['The Pianist', 'Inglourious Basterds', 'The Shawshank Redemption', 'The Godfather', 'Parasite',
          'The Dark Knight', 'Interstellar', 'Inception', 'Forrest Gump', 'City of God',
    'The Good, the Bad and the Ugly', 'Gone Girl', 'Se7en', 'The Game']

similarities = [['The Pianist', 'Inglourious Basterds'],
                ['Inglourious Basterds', 'The Shawshank Redemption'],
                ['The Shawshank Redemption', 'City of God'],
                ['The Godfather', 'Parasite'],
                ['Parasite', 'The Dark Knight'],
                ['The Dark Knight', 'Interstellar'],
                ['Interstellar', 'Inception'],
                ['Forrest Gump', 'City of God'],
                ['City of God', 'The Good, the Bad and the Ugly'],
                ['The Good, the Bad and the Ugly', 'Inglourious Basterds'],
                ['The Shawshank Redemption', 'Inglourious Basterds'],
                ['The Pianist', 'The Shawshank Redemption'],
                ['Gone Girl', 'Se7en'],
                ['Se7en', 'The Game']]

friends = [['Gone Girl', 'Se7en'],
           ['City of God', 'The Shawshank Redemption', 'Inglourious Basterds', 'Forrest Gump'],
           ['The Godfather', 'The Dark Knight', 'Parasite', 'The Pianist', 'Inception'],
           ['The Game', 'Se7en', 'The Good, the Bad and the Ugly', 'The Godfather', 'The Dark Knight', 'Forrest Gump']
          ]

### Answer is "The Godfather" or "The Dark Knight" or "Forrest Gump" or "Se7en"

First of all, let's divide the task into logical parts:

1) We must calculate Discussability ("F"). It is easy, we could use Counter from collections library, or use dict\defaultdict  with for loop.

I have used Counter with two nested loops (by friends list and movies list).

2) Then we must calculate Uniqueness ("S") - it is the mean number of similar movies that the user's friends have already seen. And we have the problem here as movies similarity have transitive property. So we need somehow to compute all similar movies.

As I can imagine there are two approaches to solve it:
* Use DFS to similarities graph traversal
* Use similarities sets: {'Parasite', '1917', 'Jojo Rabbit'}, {'Joker', 'Ford v Ferrari'}
or similarities dict: {'Parasite': 0, '1917': 0, 'Jojo Rabbit': 0, 'Joker': 1, 'Ford v Ferrari': 1}
for grouping similar movies.

I have chosen to group similar movies into the similarities dictionary by iteration over the similarities pairs in the list in for loop. And if any movie in the pair is already in similarities dict as key - we add another movie to the similarities dict keys with the same group number as his partner has in key. Otherwise, we create both dict keys with the next group number.

3) Then we should calculate "S" itself by summing up views (Discussability) of all similar films and dividing the sum by the number of all similar films for each movie in our database (list 'movies'). 

3.1) We must take into consideration the case when we need to divide by 0 - it will be 0, not infinity.

3.2) At the end we must print only one movies' name - the movie with maximum F\S. If there is a situation when we have several movies with the same F\S - the maximum function will deal with it. It will choose the first one in the direction of the passes.

And that's all.

In [10]:
### Part I --> Time: O(n*k), Space: O(n), where "n" - number of movies, "k" - number of friends
import sys
import collections
discussability = collections.Counter([film for sub in friends for film in sub]) #Space O(n)
print(f'discussability: {discussability}', end = '\n\n', file=sys.stderr)

### Part II --> Time: O(n*m), Space O(n), where "n" - number of movies, "m" - number of pairs of similar films
sim_dict = {} #Space: O(n)

idx = 0
for pair in similarities: #Time O(m)
    not_in_dict = True #Time O(1), Space: O(1)
    
    if pair[0] in sim_dict:  #Time O(n)
        sim_dict[pair[1]] = sim_dict[pair[0]] #Time O(1)
        not_in_dict = False #Time O(1)
        
    if pair[1] in sim_dict: #Time O(n)
        sim_dict[pair[0]] = sim_dict[pair[1]] #Time O(1)
        not_in_dict = False #Time O(1)
            
    if not_in_dict: #Time O(1)
        sim_dict[pair[0]] = idx #Time O(1)
        sim_dict[pair[1]] = idx #Time O(1)
        idx += 1 #Time O(1)

print(f'sim_dict: {sim_dict}', end = '\n\n', file=sys.stderr)

### Part III  Time: O(n + n*m + n), Space: O(n), where "n" - number of movies, "m" - number of pairs of similar films
watched_sim_count = {x:0 for x in movies} #Time: O(n), Space: O(n)

for film in movies: #Time O(n)
    count = 0 #Time O(1), Space: O(1)
    for x, y in sim_dict.items(): #Time O(n)
        if x != film and y == sim_dict[film]: #Time O(1)
            watched_sim_count[film] += discussability[x] #Time O(1)
            count += 1 #Time O(1)
    den = watched_sim_count[film]/count #Time O(1), Space: O(1)
    if den != 0: watched_sim_count[film] = discussability[film] / den #Time O(1)
    else: watched_sim_count[film] = float(0) #Time O(1)

print(f'watched_sim_count: {watched_sim_count}', end = '\n\n', file=sys.stderr)

print('We recommend you this movie:', max(watched_sim_count, key=lambda x: watched_sim_count[x])) #Time O(n), Space: O(n)


We recommend you this movie: 1917


discussability: Counter({'Joker': 4, '1917': 2, 'Parasite': 1, 'Jojo Rabbit': 1})

sim_dict: {'Parasite': 0, '1917': 0, 'Jojo Rabbit': 0, 'Joker': 1, 'Ford v Ferrari': 1}

watched_sim_count: {'Parasite': 0.6666666666666666, '1917': 2.0, 'Ford v Ferrari': 0.0, 'Jojo Rabbit': 0.6666666666666666, 'Joker': 0.0}



### The total time complexity of my algorithm is O(n * k + n * m + n + n * n + n) -> O(n * (k+m+1+n+1)) -> O(n * (k+m+n)),
### where "n" - number of movies, "k" - number of friends, "m" - number of pairs of similar films

### The total memory complexity of my algorithm is O(n) as it uses 3 dictionaries and some single variables,
### where "n" - number of movies